<h1><center><b>Импорт библиотек</b></center></h1>

In [ ]:
from pathlib import Path

import pandas as pd
import torch
import torch.nn.functional as F
from coca_pytorch.coca_pytorch import CoCa
from pytorch_lightning.loggers import TensorBoardLogger
from utils.dataset_coca import DataModule
from utils.utils import train_model_coca
from vit_pytorch.extractor import Extractor
from vit_pytorch.simple_vit_with_patch_dropout import SimpleViT
from utils.utils import cleanhtml
from utils.utils import lemmatize
from utils.utils import transform_tokens
from utils.utils import predict
from navec import Navec
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


<h1><center><b>Установка путей и констант</b></center></h1>

In [ ]:
# Необходимые директории
DATA_PATH = Path('data')
# директория с изображениями
IMAGES_PATH = DATA_PATH / 'images'
# директория для логов
LOG_PATH = Path('logs')
# директория для сохранения модели при обучении
CHECKPOINT_PATH = Path('checkpoints')
# Создание директорий при необходимости
DATA_PATH.mkdir(parents=True, exist_ok=True)
LOG_PATH.mkdir(parents=True, exist_ok=True)
CHECKPOINT_PATH.mkdir(parents=True, exist_ok=True)
# Текстовая модель для лемматизации
TEXT_MODEL = 'models/navec_hudlit_v1_12B_500K_300d_100q.tar'



In [ ]:
# чтение данных
data = pd.read_parquet(DATA_PATH / 'train.parquet', engine='fastparquet')
# обработка заголовков товаров (очищения, лемматизация-кодировка)
navec = Navec.load(TEXT_MODEL)
data['title'] = data.apply(lambda x: lemmatize(
    cleanhtml(x.text_fields)['title'], navec), axis=1)
data['category_id'] = data['category_id'].astype('category').cat.codes
data = data[['product_id', 'title']]
data = data[data['title'].astype(str) != '[]']
data['title'] = data.title.apply(transform_tokens)

<h1><center><b>Обучение модели</b></center></h1>

In [ ]:
# Параметры обучения:
# случайное зерно
SEED = 13
# номер видеокарты
GPU_ID = 1
# размер валидационной выборки
VAL_SIZE = 0.1
# количество эпох обучения
NUM_EPOCHS = 3
# размер батча
BATCH_SIZE = 3
# скорость обучения
LEARNING_RATE = 1e-3
# число задействованных ядер
NUM_WORKERS = 12
# количество классов
NUM_CLASSES = 874
# размер изображения для ресайза
IMAGE_SIZE = 256
# длина эмбеддинга изображения
IMAGE_EMB_LEN = 300

In [ ]:
# Убедимся, что все операции детерминированы на графическом процессоре
# (если он используется) для воспроизводимости.
torch.backends.cudnn.determinstic = True
torch.backends.cudnn.benchmark = False

# создание объекта логгирования(tensorboard)
name = 'coca'
tensor_logger = TensorBoardLogger(LOG_PATH, name=name)

# создание модуля данных
path_to_train_images = IMAGES_PATH / 'train'
datamodule = DataModule(data,
                        path_to_train_images,
                        batch_size=BATCH_SIZE,
                        num_workers=NUM_WORKERS,
                        val_size=VAL_SIZE,
                        image_size=IMAGE_SIZE,
                        seed=SEED)
# параметры модели
model_hparams = {'batch_size': BATCH_SIZE,
                 'len_image_emb': IMAGE_EMB_LEN,
                 'num_classes': NUM_CLASSES,
                 'image_size': IMAGE_SIZE
                 }
optimizer_params = {'name': 'RAdam',
                    'lr': LEARNING_RATE}
trainer = train_model_coca(
    model_hparams=model_hparams,
    optimizer_params=optimizer_params,
    ckpt_path=None,
    logger=tensor_logger,
    scheduler=None,
    checkpoint_path=CHECKPOINT_PATH / f'{name}_{optimizer_params["name"]}',
    device=device,
    gpu_id=[GPU_ID],
    num_epochs=NUM_EPOCHS,
    data_module=datamodule,
    seed=SEED,
    unfreeze_epoch=None
)
